# GCP Play with LLMs

Ref: https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/text-bison


In [5]:
import numpy as np
import pandas as pd
import json
import time

#from google.colab import auth as google_auth
#google_auth.authenticate_user()

import vertexai
#from vertexai.preview.language_models import TextGenerationModel, TextEmbeddingModel
from vertexai.language_models import TextGenerationModel, TextEmbeddingModel


2024-03-07 15:55:09.305570: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-07 15:55:15.386834: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-03-07 15:55:15.386969: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-03-07 15:55:15.386979: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

## Project ID and Region

For multiple projects, do:
```
PROJECT_ID = "[your-project-id]"

# Set the project id
! gcloud config set project {PROJECT_ID}
```

In [6]:
PROJECT_ID = ! gcloud config get core/project
PROJECT_ID = PROJECT_ID[0]

REGION = "europe-west2"

PROJECT_ID, REGION

('playpen-714696', 'europe-west2')

## Text Generation

### Call from terminal using

In [7]:
%env PROJECT_ID=$PROJECT_ID

env: PROJECT_ID=playpen-714696


### Vertex AI SDK for Python

In [10]:

def interview(temperature: float = .2) -> None:
    """Ideation example with a Large Language Model"""

    # TODO developer - override these parameters as needed:
    parameters = {
        "temperature": temperature,
        "max_output_tokens": 256,   
        "top_p": .8,                
        "top_k": 40,                 
    }

    model = TextGenerationModel.from_pretrained("text-bison@001")
    response = model.predict(
        'Give me ten interview questions for the role of program manager.',
        **parameters,
    )
    print(f"Response from Model: \n{response.text}")
    

In [11]:
interview()

Response from Model: 
1. What is your experience with project management?
2. What are your strengths and weaknesses as a project manager?
3. How do you handle conflict?
4. How do you communicate with stakeholders?
5. How do you stay organized?
6. How do you manage your time?
7. How do you deal with stress?
8. What are your goals for the future?
9. Why do you want to work for our company?
10. What are your salary expectations?


In [13]:
model = TextGenerationModel.from_pretrained("text-bison@001")

In [14]:
prompt = "Who was the first elephant to visit the moon?"
#prompt = "Who was the first elephant to visit the moon? I think it was called Lara"

print(
    model.predict(prompt=prompt, max_output_tokens=256).text
)

The first elephant to visit the moon was Luna. Luna was a female elephant who was born in 1963 at the San Diego Zoo. In 1968, she was selected to be the first elephant to visit the moon. Luna was trained to wear a spacesuit and to ride in a rocket. She was also trained to perform various tasks on the moon, such as collecting samples of moon rocks and soil.

Luna's trip to the moon was a success. She spent several days on the moon, collecting samples and performing experiments. She also became the first animal to walk on the moon. Luna's trip to the moon was a major scientific achievement and helped to pave the way for future human missions to the moon.

Luna returned to Earth in 1969. She was a national hero and was celebrated by people all over the world. Luna lived a long and happy life, dying in 2003 at the age of 40.


### Zero shot prompt

In [15]:
prompt = """Decide whether a Tweet's sentiment is positive, neutral, or negative.

Tweet: I loved the new YouTube video you made!
Sentiment:
"""

print(model.predict(prompt=prompt, max_output_tokens=256).text)

positive


## Read the company information  

In [133]:
compnay_info_file = "../fos_complaints_company_2013.csv"
UK_banks = [
    "Barclays Bank UK PLC"
]

In [134]:
company_df = pd.read_csv(compnay_info_file, sep=";")
company_df.head()

,company,drn
0,Omni Capital Retail Finance Limited,DRN-2610030
1,@SIPP LIMITED,DRN-2796286
2,Retail Money Market Ltd,DRN-2815239
3,CMC Spreadbet Plc,DRN-2849730
4,AJ Bell Management Limited,DRN-2864928


In [140]:
#df[df["company"].str.contains("Bank|Insurance|Barclays|Lloyds|HSBC|Santander|Westminster|Nationwide|Admiral|Aviva", case=False)] # 14633 rows

#df[df["company"].str.contains("Barclays|Lloyds|HSBC|Santander|Westminster|Nationwide|Admiral|Aviva", case=False)] # 5694 rows

company_df = company_df[company_df["company"].str.contains("Lloyds Bank PLC", case=False)]#.sample(5) # 810 rows
company_df.head()

,company,drn
42,Lloyds Bank PLC,DRN-3174532
88,Lloyds Bank PLC,DRN-3300784
112,Lloyds Bank PLC,DRN-3322403
145,Lloyds Bank PLC,DRN-3378564
306,Lloyds Bank PLC,DRN-3501539


## Read the complaint file information

In [ ]:
from google.cloud import storage


def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
    # bucket_name = "your-bucket-name"

    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name)
    
    return blobs
#    return [blob.name for blob in blobs]
        
def list_blobs_with_prefix(bucket_name, prefix, delimiter=None):
    """Lists all the blobs in the bucket that begin with the prefix.

    This can be used to list all blobs in a "folder", e.g. "public/".

    The delimiter argument can be used to restrict the results to only the
    "files" in the given "folder". Without the delimiter, the entire tree under
    the prefix is returned. For example, given these blobs:

        a/1.txt
        a/b/2.txt

    If you specify prefix ='a/', without a delimiter, you'll get back:

        a/1.txt
        a/b/2.txt

    However, if you specify prefix='a/' and delimiter='/', you'll get back
    only the file directly under 'a/':

        a/1.txt

    As part of the response, you'll also get back a blobs.prefixes entity
    that lists the "subfolders" under `a/`:

        a/b/
    """

    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix, delimiter=delimiter)

    return blobs
#    return [blob.name for blob in blobs]




In [ ]:
#[b.name for b in list_blobs("fos-trend-bucket")]
[b.name for b in list_blobs_with_prefix(bucket_name="fos-trend-bucket", prefix="text_extracts")]

In [136]:
#[b.name for b in list_blobs_with_prefix(bucket_name="fos-trend-bucket", prefix="text_extracts") if b.name in a["drn"] ]

blob_df = pd.DataFrame(
    {
        "blob": [b for b in list_blobs_with_prefix(bucket_name="fos-trend-bucket", prefix="text_extracts")],
        "drn": [b.name[14:25] for b in list_blobs_with_prefix(bucket_name="fos-trend-bucket", prefix="text_extracts")],
    }
)

blob_df.head()

,blob,drn
0,"<Blob: fos-trend-bucket, text_extracts/DRN-100...",DRN-1000311
1,"<Blob: fos-trend-bucket, text_extracts/DRN-100...",DRN-1000423
2,"<Blob: fos-trend-bucket, text_extracts/DRN-317...",DRN-3174532
3,"<Blob: fos-trend-bucket, text_extracts/DRN-330...",DRN-3300784
4,"<Blob: fos-trend-bucket, text_extracts/DRN-337...",DRN-3378564


In [143]:
selected_df = blob_df.merge(company_df, how="inner", on='drn')

selected_df.head()

,blob,drn,company
0,"<Blob: fos-trend-bucket, text_extracts/DRN-317...",DRN-3174532,Lloyds Bank PLC
1,"<Blob: fos-trend-bucket, text_extracts/DRN-330...",DRN-3300784,Lloyds Bank PLC
2,"<Blob: fos-trend-bucket, text_extracts/DRN-337...",DRN-3378564,Lloyds Bank PLC
3,"<Blob: fos-trend-bucket, text_extracts/DRN-452...",DRN-4527390,Lloyds Bank PLC


## Read the complaint text

In [198]:
from io import StringIO

def read_txt(blob):
    #for b in list_blobs_with_prefix(bucket_name="fos-trend-bucket", prefix="text_extracts")]
    f = StringIO(blob.download_as_text(encoding="utf-8"))
    
    return f.read()


#print(read_txt(selected_df["blob"].iloc[0]))

In [148]:
for index, row in selected_df.iterrows():
    print(row['drn'], row['company'])
    print(read_txt(row["blob"])[0:20])

DRN-3174532 Lloyds Bank PLC
﻿company;drn
Omni C
DRN-3300784 Lloyds Bank PLC
﻿company;drn
Omni C
DRN-3378564 Lloyds Bank PLC
﻿company;drn
Omni C
DRN-4527390 Lloyds Bank PLC
﻿company;drn
Omni C


In [192]:
import vertexai
from vertexai.language_models import TextGenerationModel
vertexai.init(project="playpen-714696", location="europe-west4")


def processing_doc(doc_content) -> None:
    """Get one row for the document"""

    parameters = {
        "candidate_count": 1,
        "max_output_tokens": 2048,
        "temperature": 0,
        "top_p": 1
    }

#    model = TextGenerationModel.from_pretrained("text-bison@001")
    model = TextGenerationModel.from_pretrained("text-bison")
    
    prompt = f"""You are a text processing agent working with Financial Ombudsman Service (FOS) decision document.
Extract specified values from the source text. 
Return answer as JOSN object with following fields:
 - \"Case number\" <number>
 - \"Complainant\" <string>
 - \"Defendant\" <string>
 - \"Defendant\'s industry\" <string>
 - \"What was the complainant complaining about\" <string>
 - \"What product involved in the complaint\" <string>
 - \"Three key words on this complaint\" <string>
 - \"Three key topics on this complaint\" <string>
 - \"Claimed value in pounds\" <number> 
 - \"When did it happen? (date only)\" <date>
 - \"Final decision (uphold or opposite)\" <string>
 - \"Ombudsman\'s name\" <string>
 - \"Decision deadline\" <date>
 - \"what was the complaint relating to\" <string>
 - \"what was the main reason that the complainant gave for them complaining\"
 - \"What did the complaint want\" <string>
 - \"Summary of what happened\" <string>
 - \"Summary of the Ombudsman\'s reasoning\" <string>


Do not infer any data based on previous training, strictly use only source text given below as input.
========
{doc_content}
========
\"\"\"
"""

    response = model.predict(
        prompt,
        **parameters
    )
   
    return response.text



In [194]:
import json

doc_content = read_txt(selected_df["blob"].iloc[0])

response_string = processing_doc({doc_content[0:1000]})

cleaned_json_string = response_string.replace('```','').split("json",1)[1]

d = json.loads(cleaned_json_string)
print(d)

df = pd.json_normalize(d)
df

{'Case number': None, 'Complainant': None, 'Defendant': None, "Defendant's industry": None, 'What was the complainant complaining about': None, 'What product involved in the complaint': None, 'Three key words on this complaint': None, 'Three key topics on this complaint': None, 'Claimed value in pounds': None, 'When did it happen? (date only)': None, 'Final decision (uphold or opposite)': None, "Ombudsman's name": None, 'Decision deadline': None, 'what was the complaint relating to': None, 'what was the main reason that the complainant gave for them complaining': None, 'What did the complaint want': None, 'Summary of what happened': None, "Summary of the Ombudsman's reasoning": None}


,Case number,Complainant,Defendant,Defendant's industry,What was the complainant complaining about,What product involved in the complaint,Three key words on this complaint,Three key topics on this complaint,Claimed value in pounds,When did it happen? (date only),Final decision (uphold or opposite),Ombudsman's name,Decision deadline,what was the complaint relating to,what was the main reason that the complainant gave for them complaining,What did the complaint want,Summary of what happened,Summary of the Ombudsman's reasoning
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [213]:



# doc_content = read_txt(selected_df["blob"].iloc[0])

# response_string = processing_doc({doc_content[0:1000]})

# cleaned_json_string = response_string.replace('```','').split("json",1)[1]

# d = json.loads(cleaned_json_string)
# print(d)

# df = pd.json_normalize(d)
# df


#[read_text(r["blob"]) for row in selected_df.iterrows()]

df_list = []
for index, (blob, drn, company) in selected_df.iterrows():
    doc_content = read_txt(blob)
    response_string = processing_doc({doc_content[0:1000]})
    cleaned_json_string = response_string.replace('```','').split("json",1)[1]
    d = json.loads(cleaned_json_string)
    df = pd.json_normalize(d)
    df_list.append(df)

df_list
    

[  Case number Complainant Defendant Defendant's industry  \
 0        None        None      None                 None   
 
   What was the complainant complaining about  \
 0                                       None   
 
   What product involved in the complaint Three key words on this complaint  \
 0                                   None                              None   
 
   Three key topics on this complaint Claimed value in pounds  \
 0                               None                    None   
 
   When did it happen? (date only) Final decision (uphold or opposite)  \
 0                            None                                None   
 
   Ombudsman's name Decision deadline what was the complaint relating to  \
 0             None              None                               None   
 
   what was the main reason that the complainant gave for them complaining  \
 0                                               None                        
 
   What did the compla

In [215]:
summary_table = pd.concat(df_list, axis=0)

In [218]:
summary_table_name = "../output/summary_lloyds.csv"

summary_table.to_csv(summary_table_name, index=False)
